En SQL, Cascade Delete Relationships es una característica que asegura que, cuando se elimina un registro en una tabla "principal", todos los registros relacionados en la tabla "hija" también se eliminen automáticamente. Esto es útil para mantener la integridad referencial y evitar datos huérfanos (registros en la tabla hija que ya no tienen una relación válida con un registro en la tabla principal).

In [1]:
from sqlmodel import SQLModel, Field, create_engine, Session, Relationship
from typing import List, Optional
from datetime import date

Se va a crear una Tabla que relacione un experimento en termodinamica, en la suponemos que estamos modelando datos. Tenemos dos tablas principales:

Experimentos (Tabla principal): Registra los experimentos realizados.
Medidas (Tabla hija): Contiene las mediciones realizadas para cada experimento, como presión, temperatura, volumen, etc.

Cuando se elimina un experimento de la tabla Experiments, queremos que todas las mediciones asociadas en Measurements también se eliminen automáticamente. Es en esencia el ejemplo

In [2]:
class Experimento(SQLModel, table=True):
    __tablename__ = 'experimentos'  # Nombre de la tabla 1
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    descripcion: Optional[str]
    fecha_realizada: date
    medidas: List["Medidas"] = Relationship(
        back_populates="experimento",
        sa_relationship_kwargs={"cascade": "all, delete-orphan"}
    )

class Medidas(SQLModel, table=True):
    __tablename__ = 'medidas'  # Nombre de la tabla 2
    id: Optional[int] = Field(default=None, primary_key=True)
    experimento_id: int = Field(foreign_key="experimentos.id", nullable=False)
    temperatura: float  # Temperatura en Kelvin
    presion: float     # Presión en Pascales
    volumen: float     # Volumen en m³
    experimento: "Experimento" = Relationship(back_populates="medidas")

In [3]:
# Crear la base de datos
engine = create_engine("sqlite:///thermodynamics.db", echo=True)
SQLModel.metadata.create_all(engine)

2024-11-22 14:53:10,563 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-22 14:53:10,727 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("experimentos")
2024-11-22 14:53:10,731 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-22 14:53:10,734 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("medidas")
2024-11-22 14:53:10,735 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-22 14:53:10,739 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
# Las tablas se crearon en el orden adecuado=
SQLModel.metadata.create_all(engine)

2024-11-22 14:53:14,423 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-22 14:53:14,424 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("experimentos")
2024-11-22 14:53:14,426 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-22 14:53:14,430 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("medidas")
2024-11-22 14:53:14,431 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-22 14:53:14,435 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
# Ejemplo de inserción de nuevos datos
with Session(engine) as session:
    # Crear un nuevo experimento con mediciones
    experimento1 = Experimento(
        name="Expansión isotérmica",
        descripcion="Estudio de la expansión isotérmica de un gas ideal",
        fecha_realizada=date(2024, 11, 15),
        medidas=[
            Medidas(temperatura=350.15, presion=150000, volumen=0.1),
            Medidas(temperatura=360.15, presion=145000, volumen=0.12)
        ]
    )
    session.add(experimento1)

        # Crear otro experimento con mediciones
    experimento2 = Experimento(
        name="Calor específico",
        descripcion="Determinación del calor específico de un gas a presión constante",
        fecha_realizada=date(2024, 11, 16),
        medidas=[
            Medidas(temperatura=310.15, presion=200000, volumen=0.08),
            Medidas(temperatura=320.15, presion=195000, volumen=0.09)
        ]
    )
    session.add(experimento2)

In [6]:
# Ejemplo de inserción y eliminación
with Session(engine) as session:
    # Crear un experimento con mediciones
    experimento = Experimento(
        name="Compresión adiabática",
        descripcion="Estudio de la compresión adiabática de un gas ideal",
        fecha_realizada=date(2024, 11, 1),  # Objeto date de Python (acá habia un error)
        medidas=[
            Medidas(temperatura=300.15, presion=202650, volumen=0.05),
            Medidas(temperatura=400.15, presion=405300, volumen=0.025)
        ]
    )
    session.add(experimento)
    session.commit()

    # Eliminar el experimento y las mediciones asociadas
    session.delete(experimento)
    session.commit()

2024-11-22 14:53:22,353 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-22 14:53:22,358 INFO sqlalchemy.engine.Engine INSERT INTO experimentos (name, descripcion, fecha_realizada) VALUES (?, ?, ?)
2024-11-22 14:53:22,359 INFO sqlalchemy.engine.Engine [generated in 0.00185s] ('Compresión adiabática', 'Estudio de la compresión adiabática de un gas ideal', '2024-11-01')
2024-11-22 14:53:22,367 INFO sqlalchemy.engine.Engine INSERT INTO medidas (experimento_id, temperatura, presion, volumen) VALUES (?, ?, ?, ?) RETURNING id
2024-11-22 14:53:22,368 INFO sqlalchemy.engine.Engine [generated in 0.00044s (insertmanyvalues) 1/2 (ordered; batch not supported)] (1, 300.15, 202650.0, 0.05)
2024-11-22 14:53:22,370 INFO sqlalchemy.engine.Engine INSERT INTO medidas (experimento_id, temperatura, presion, volumen) VALUES (?, ?, ?, ?) RETURNING id
2024-11-22 14:53:22,371 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] (1, 400.15, 405300.0, 0.025)
2024-11-22 14:53